In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns

# Any results you write to the current directory are saved as output.

In [2]:
train_data = pd.read_csv('train_modified.csv') # Important!!
print(train_data.shape)
train_data.head()

/home/tintash/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(903653, 56)


,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [3]:
train_data.columns.values

array(['Unnamed: 0', 'channelGrouping', 'date', 'fullVisitorId',
       'sessionId', 'socialEngagementType', 'visitId', 'visitNumber',
       'visitStartTime', 'device.browser', 'device.browserSize',
       'device.browserVersion', 'device.deviceCategory',
       'device.flashVersion', 'device.isMobile', 'device.language',
       'device.mobileDeviceBranding', 'device.mobileDeviceInfo',
       'device.mobileDeviceMarketingName', 'device.mobileDeviceModel',
       'device.mobileInputSelector', 'device.operatingSystem',
       'device.operatingSystemVersion', 'device.screenColors',
       'device.screenResolution', 'geoNetwork.city', 'geoNetwork.cityId',
       'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.latitude', 'geoNetwork.longitude', 'geoNetwork.metro',
       'geoNetwork.networkDomain', 'geoNetwork.networkLocation',
       'geoNetwork.region', 'geoNetwork.subContinent', 'totals.bounces',
       'totals.hits', 'totals.newVisits', 'totals.pageviews',
       'tota

In [8]:
train_df = train_data.sample(frac=0.8,random_state=200)
test_df = train_data.drop(train_df.index)

In [ ]:
relevantAttributes = ['visitNumber', 'date', 'fullVisitorId', 'channelGrouping',
#                       'device.deviceCategory',
                      'device.operatingSystem', 'device.browser',
                       'geoNetwork.continent', 'geoNetwork.country',
                      'totals.hits', 'totals.pageviews', 
                      'totals.transactionRevenue']
#                        'fullVisitorId']
#                       'year', 'month', 'weekday',
#                       'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']

train_data = train_data[relevantAttributes]

In [5]:
train_Y = train_df['totals.transactionRevenue']
train_X = train_df.drop(labels=['totals.transactionRevenue'],axis=1)

In [ ]:
y_trainTrue_class = train_Y.copy()
y_trainTrue_class[y_trainTrue_class>0]=1

In [8]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

regr_model = DecisionTreeRegressor(max_depth=2)
regr_model.fit(train_X, train_Y)

ValueError: could not convert string to float: 'google'

In [1]:
import numpy as np 
import pandas as pd 
import json

from pandas.io.json import json_normalize
import seaborn as sns 

import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.metrics import mean_squared_error
import functools
from multiprocessing import Pool
import logging
import gc

  
NUM_ROUNDS = 20000
VERBOSE_EVAL = 250
STOP_ROUNDS = 250
N_SPLITS = 9
TIME_SPLIT = True


def process_date_time(data_df):
#     logger.info('Start date')
    data_df['date'] = data_df['date'].astype(str)
    data_df["date"] = data_df["date"].apply(lambda x : x[:4] + "-" + x[4:6] + "-" + x[6:])
    data_df["date"] = pd.to_datetime(data_df["date"])   
    data_df["year"] = data_df['date'].dt.year
    data_df["month"] = data_df['date'].dt.month
    data_df["day"] = data_df['date'].dt.day
    data_df["weekday"] = data_df['date'].dt.weekday
    data_df['weekofyear'] = data_df['date'].dt.weekofyear
    data_df['month_unique_user_count'] = data_df.groupby('month')['fullVisitorId'].transform('nunique')
    data_df['day_unique_user_count'] = data_df.groupby('day')['fullVisitorId'].transform('nunique')
    data_df['weekday_unique_user_count'] = data_df.groupby('weekday')['fullVisitorId'].transform('nunique')
    data_df['weekofyear_unique_user_count'] = data_df.groupby('weekofyear')['fullVisitorId'].transform('nunique')
#     logger.info('Done with date')
    
    return data_df

def process_format(data_df):
#     logger.info('Start format')
    for col in ['visitNumber', 'totals.hits', 'totals.pageviews']:
        data_df[col] = data_df[col].astype(float)
    data_df['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True)
    data_df['trafficSource.isTrueDirect'].fillna(False, inplace=True)
#     logger.info('Done with format')
    return data_df
    
def process_device(data_df):
#     logger.info('Start device')
    data_df['browser_category'] = data_df['device.browser'] + '_' + data_df['device.deviceCategory']
    data_df['browser_os'] = data_df['device.browser'] + '_' + data_df['device.operatingSystem']
    data_df['category_os'] = data_df['device.deviceCategory'] + '_' + data_df['device.operatingSystem']
    data_df['visits_id_browser_mean'] = np.log1p(data_df.groupby(['device.browser'])['visitNumber'].transform('mean'))
    data_df['visits_id_os_mean'] = np.log1p(data_df.groupby(['device.operatingSystem'])['visitNumber'].transform('mean'))
    data_df['visits_id_cat_mean'] = np.log1p(data_df.groupby(['device.deviceCategory'])['visitNumber'].transform('mean'))
    data_df['browser_unique_user_count'] = data_df.groupby('device.browser')['fullVisitorId'].transform('nunique')
    data_df['os_unique_user_count'] = data_df.groupby('device.operatingSystem')['fullVisitorId'].transform('nunique')
#     logger.info('Done with device')
    return data_df

def process_totals(data_df):
#     logger.info('Start totals')
    #data_df['visitNumber'] = data_df['visitNumber']
    data_df['visits_id_sum'] = data_df.groupby(['fullVisitorId'])['visitNumber'].transform('sum')
    data_df['visits_id_min'] =  data_df.groupby(['fullVisitorId'])['visitNumber'].transform('min')
    data_df['visits_id_max'] = data_df.groupby(['fullVisitorId'])['visitNumber'].transform('max')
    data_df['visits_id_mean'] = data_df.groupby(['fullVisitorId'])['visitNumber'].transform('mean')
    data_df['visits_id_nunique'] = data_df.groupby('visitNumber')['fullVisitorId'].transform('nunique')
    #data_df['totals_hits'] = data_df['totals_hits']
    data_df['hits_id_sum'] = data_df.groupby(['fullVisitorId'])['totals.hits'].transform('sum')
    data_df['hits_id_cnt'] = data_df.groupby(['fullVisitorId'])['totals.hits'].transform('count')
    data_df['totals_pageviews'] = data_df['totals.pageviews'].fillna(0)
    data_df['pageviews_id_sum'] = data_df.groupby(['fullVisitorId'])['totals.pageviews'].transform('sum')
    data_df['pageviews_id_cnt'] = data_df.groupby(['fullVisitorId'])['totals.pageviews'].transform('count')
    data_df['mean_hits_per_day'] = data_df.groupby(['day'])['totals.hits'].transform('mean')
    data_df['sum_hits_per_day'] = data_df.groupby(['day'])['totals.hits'].transform('sum')
    data_df['max_hits_per_day'] = data_df.groupby(['day'])['totals.hits'].transform('max')
    data_df['min_hits_per_day'] = data_df.groupby(['day'])['totals.hits'].transform('min')
    data_df['var_hits_per_day'] = data_df.groupby(['day'])['totals.hits'].transform('var')
    data_df['mean_pageviews_per_day'] = data_df.groupby(['day'])['totals.pageviews'].transform('mean')
    data_df['sum_pageviews_per_day'] = data_df.groupby(['day'])['totals.pageviews'].transform('sum')
    data_df['max_pageviews_per_day'] = data_df.groupby(['day'])['totals.pageviews'].transform('max')
    data_df['min_pageviews_per_day'] = data_df.groupby(['day'])['totals.pageviews'].transform('min')   
    
#     logger.info('Done with totals')
    return data_df

def process_geo_network(data_df):
#     logger.info('Start geoNetworks')
    data_df['mean_visit_per_network_domain']= data_df.groupby('geoNetwork.networkDomain')['visitNumber'].transform('mean')
    data_df['min_visit_per_network_domain']= data_df.groupby('geoNetwork.networkDomain')['visitNumber'].transform('min')
    data_df['max_visit_per_network_domain']= data_df.groupby('geoNetwork.networkDomain')['visitNumber'].transform('max')
    data_df['sum_pageviews_per_network_domain'] = data_df.groupby('geoNetwork.networkDomain')['totals_pageviews'].transform('sum')
    data_df['count_pageviews_per_network_domain'] = data_df.groupby('geoNetwork.networkDomain')['totals.pageviews'].transform('count')
    data_df['mean_pageviews_per_network_domain'] = data_df.groupby('geoNetwork.networkDomain')['totals.pageviews'].transform('mean')
    data_df['max_pageviews_per_network_domain'] = data_df.groupby('geoNetwork.networkDomain')['totals.pageviews'].transform('max')
    data_df['sum_hits_per_network_domain'] = data_df.groupby('geoNetwork.networkDomain')['totals.hits'].transform('sum')
    data_df['count_hits_per_network_domain'] = data_df.groupby('geoNetwork.networkDomain')['totals.hits'].transform('count')
    data_df['mean_hits_per_network_domain'] = data_df.groupby('geoNetwork.networkDomain')['totals.hits'].transform('mean')
    data_df['max_hits_per_network_domain'] = data_df.groupby('geoNetwork.networkDomain')['totals.hits'].transform('max')
    data_df['network_domain_id_nunique'] = data_df.groupby('geoNetwork.networkDomain')['fullVisitorId'].transform('nunique')
#     logger.info('Done with geoNetworks')
    return data_df

def process_traffic_source(data_df):
#     logger.info('Start trafficSource')
    data_df['source_country'] = data_df['trafficSource.source'] + '_' + data_df['geoNetwork.country']
    data_df['campaign_medium'] = data_df['trafficSource.campaign'] + '_' + data_df['trafficSource.medium']
    data_df['campaign_country'] = data_df['trafficSource.campaign'] + '_' + data_df['geoNetwork.country']
    data_df['medium_hits_mean'] = data_df.groupby(['trafficSource.medium'])['totals.hits'].transform('mean')
    data_df['medium_hits_max'] = data_df.groupby(['trafficSource.medium'])['totals.hits'].transform('max')
    data_df['medium_hits_min'] = data_df.groupby(['trafficSource.medium'])['totals.hits'].transform('min')
    data_df['medium_hits_sum'] = data_df.groupby(['trafficSource.medium'])['totals.hits'].transform('sum')
    data_df['medium_hits_var'] = data_df.groupby(['trafficSource.medium'])['totals.hits'].transform('var')
#     logger.info('Done with trafficSource')
    return data_df

def drop_convert_columns(train_df=None, test_df=None):
#     logger.info('Start drop convert')
    cols_to_drop = [col for col in train_df.columns if train_df[col].nunique(dropna=False) == 1]
    train_df.drop(cols_to_drop, axis=1, inplace=True)
    test_df.drop([col for col in cols_to_drop if col in test_df.columns], axis=1, inplace=True)
    ###only one not null value
    train_df.drop(['trafficSource.campaignCode'], axis=1, inplace=True)
    ###converting columns format
    train_df['totals.transactionRevenue'] = train_df['totals.transactionRevenue'].astype(float)
    train_df['totals.transactionRevenue'] = train_df['totals.transactionRevenue'].fillna(0)
    train_df['totals.transactionRevenue'] = np.log1p(train_df['totals.transactionRevenue'])
#     logger.info('Done with  drop convert')
    return train_df, test_df
    
    
def process_categorical_columns(train_df=None, test_df=None):
    ## Categorical columns
#     logger.info('Process categorical columns ...')
    num_cols = ['month_unique_user_count', 'day_unique_user_count', 'weekday_unique_user_count', 'weekofyear_unique_user_count',
                'visits_id_browser_mean', 'visits_id_os_mean','visits_id_cat_mean',
                'visitNumber',  'visits_id_sum', 'visits_id_min', 'visits_id_max', 'visits_id_mean', 'visits_id_nunique',
                'browser_unique_user_count', 'os_unique_user_count',
                'totals_hits', 'hits_id_sum', 'hits_id_cnt',
                'totals_pageviews', 'pageviews_id_sum', 'pageviews_id_cnt',
                'mean_visit_per_network_domain','min_visit_per_network_domain','max_visit_per_network_domain',
                'mean_hits_per_day', 'sum_hits_per_day', 'min_hits_per_day', 'max_hits_per_day', 'var_hits_per_day',
                'mean_pageviews_per_day', 'sum_pageviews_per_day', 'min_pageviews_per_day', 'max_pageviews_per_day',
                'sum_pageviews_per_network_domain', 'count_pageviews_per_network_domain', 'mean_pageviews_per_network_domain','max_pageviews_per_network_domain',
                'sum_hits_per_network_domain', 'count_hits_per_network_domain', 'mean_hits_per_network_domain', 'max_hits_per_network_domain', 'network_domain_id_nunique',
                'medium_hits_mean','medium_hits_min','medium_hits_max','medium_hits_sum', 'medium_hits_var']
    
    not_used_cols = ["date", "fullVisitorId", "sessionId", 
            "visitId", "visitStartTime", 'totals.transactionRevenue', 'trafficSource.referralPath']
    cat_cols = [col for col in train_df.columns if col not in num_cols and col not in not_used_cols]
    for col in cat_cols:
#         logger.info('Process categorical columns:{}'.format(col))
        lbl = LabelEncoder()
        lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
        train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
        test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))
    return not_used_cols, train_df, test_df

def model(train_df=None, test_df=None,not_used_cols=None):
#     logger.info('Start prepare model')
    train_df = train_df.sort_values('date')
    X_test = test_df.drop([col for col in not_used_cols if col in test_df.columns], axis=1)
    
    ## Model parameters
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 20180926,
        "verbosity" : -1
    }
   
    ## Model
#     logger.info('Start traininig model')
    model = lgb.LGBMRegressor(**params, n_estimators = NUM_ROUNDS, nthread = 4, n_jobs = -1)
    
    prediction = np.zeros(test_df.shape[0])
    
    if(TIME_SPLIT):
        import datetime
        train = train_df[train_df['date']<=datetime.date(2017,6,15)]
        valid = train_df[train_df['date']>datetime.date(2017,6,15)]
        X_train = train.drop(not_used_cols, axis=1)
        y_train = train['totals.transactionRevenue']
        X_valid = valid.drop(not_used_cols, axis=1)
        y_valid = valid['totals.transactionRevenue']
        model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse',
                    verbose=VERBOSE_EVAL, early_stopping_rounds=STOP_ROUNDS)
        prediction = model.predict(X_test, num_iteration=model.best_iteration_)           
    else:
        X = train_df.drop(not_used_cols, axis=1)
        y = train_df['totals.transactionRevenue']
        folds = KFold(n_splits=N_SPLITS, shuffle=True, random_state=20180917)
        for fold_n, (train_index, test_index) in enumerate(folds.split(X)):
            print(fold_n)
            X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
            
            model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse',
                    verbose=VERBOSE_EVAL, early_stopping_rounds=STOP_ROUNDS)
            
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            prediction += y_pred
            print('Done with Fold:{}'.format(fold_n))
        prediction /= N_SPLITS
    
    return prediction

    
def main(sum_of_logs=False, nrows=None):
    #Feature processing
    ## Load data

    train_data = pd.read_csv('train_modified.csv') # Important!!
    train_df = train_data.sample(frac=0.8,random_state=200)
    test_df = train_data.drop(train_df.index)


    train_df = train_df
    train_df = process_date_time(train_df)
    test_df = test_df
    test_df = process_date_time(test_df)
    
    ## Drop columns
    train_df, test_df = drop_convert_columns(train_df, test_df)
    
    ## Features engineering
    train_df = process_format(train_df)
    train_df = process_device(train_df)
    train_df = process_totals(train_df)
    train_df = process_geo_network(train_df)
    train_df = process_traffic_source(train_df)
    
    test_df = process_format(test_df)
    test_df = process_device(test_df)
    test_df = process_totals(test_df)
    test_df = process_geo_network(test_df)
    test_df = process_traffic_source(test_df)
    
    ## Categorical columns
    not_used_cols, train_df, test_df = process_categorical_columns(train_df, test_df)
    
    # Model
    prediction = model(train_df, test_df, not_used_cols)
    
    #Submission
    submission(test_df, prediction)
    

if __name__ == "__main__":
    main(nrows=None)

/home/tintash/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:191: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:192: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.


Training until validation scores don't improve for 250 rounds.
[250]	training's rmse: 1.478	valid_1's rmse: 1.66632
[500]	training's rmse: 1.40954	valid_1's rmse: 1.66707
Early stopping, best iteration is:
[384]	training's rmse: 1.43964	valid_1's rmse: 1.66569


ValueError: Number of features of the model must match the input. Model n_features_ is 85 and input n_features is 86 